In [109]:
## CPMSC 448 Homework 4
### Problem 2 - Boosted Decision Trees

#### Name: Ashley Gallagher
#### PSU id: ang5456

In [110]:
import numpy as np
import pandas as pd
pd.set_option('display.max.columns', 100)
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.datasets import load_svmlight_file
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [111]:
train_data = load_svmlight_file('a9a.txt')
X_train = train_data[0]
Y_train = train_data[1]
test_data = load_svmlight_file('a9a.t')
X_test = test_data[0]
Y_test = test_data[1]

In [112]:
#x_train, x_test, y_train , y_test = train_test_split(X_train, Y_train, test_size=.3, random_state=163)
model = XGBClassifier()
model.fit(X_train, Y_train)
print(model)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)


In [113]:
#tuning hyper parameters
n_estimators = [50, 100, 200, 300]
max_depth = [3, 6, 9]
lam = [1.0, 0.0]
learning_rate = [.1, .2, .3,]
missing = [None, 0]
objective = ('binary:logistic', 'binary:logitraw', 'binary:hinge')
parameters = []
for ne in n_estimators:
    for md in max_depth:
        for l in lam:
            for lr in learning_rate:
                for m in missing:
                    for o in objective:
                        parameters.append([ne, md, l, lr, m, o])

In [114]:
#use XGB's cross validation to tune
new_y = []
for t in Y_train:
    if t == -1:
        new_y.append(0)
    else:
        new_y.append(t)
best = 0
best_params = []
iter = 0
for p in parameters:
    iter += 1
    print('round {0}'.format(iter))
    params = {
        'n_estimators': p[0],
        'max_depth': p[1],
        'lambda': p[2],
        'learning_rate': p[3],
        'missing': p[4],
        'objective': p[5],
    }
    model = XGBClassifier(
        n_estimators= p[0],
        max_depth= p[1],
        reg_lambda=p[2],
        learning_rate=p[3],
        missing= p[4],
        objective= p[5],
    )
    kfold = KFold(n_splits=10, random_state=7)
    results = cross_val_score(model, X_train, new_y, cv=kfold)
    accuracy = results.mean()*100
    print(accuracy)
    if (accuracy > best):
        best = accuracy
        best_model = params

print(best_model)
print(best)

round 1
84.19583807018563
round 2


KeyboardInterrupt: 

In [108]:
#Final Claasifier and metrics
final_model = XGBClassifier(
        n_estimators= 300,
        max_depth= 3,
        reg_lambda= 1.0,
        learning_rate= .2,
        missing= None,
        objective= 'binary:logistic',
    )

kfold = KFold(n_splits=5, random_state=7)
results = cross_val_score(final_model, X_train, new_y, cv=kfold)

print('Cross Validation Training Error Rate for final Model: {0}'.format(1-results.mean()))

final_model.fit(X_train, Y_train)

print('Test Error Rate for the final Model: {0}'.format(1-final_model.score(X_test, Y_test)))

Cross Validation Training Error Rate for final Model: 0.14965747959759912
Test Error Rate for the final Model: 0.14710398624163135
